# Qwen critique → Llama reward walkthrough

이 노트북은 `examples/grpo_trainer/run_qwen2.5-7b_instruct_critique.sh` 흐름을 노트북 형태로 옮겨,
1) Qwen2.5-7B-Instruct로 critique를 생성하고,
2) vLLM으로 띄운 Llama reward 모델에 critique를 넣어 variant 문제를 풀고,
3) 변형 문제별 정답률과 출력 내용을 확인하는 과정을 보여줍니다.

GPU 자원이 필요하므로 실행 전 리소스를 확인하세요.


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from pathlib import Path
import os
import json
import pandas as pd
import sys

# 경로 설정
REPO_ROOT = Path.cwd().resolve()
if not (REPO_ROOT / "verl").exists():
    # 노트북 위치에 따라 repo root 탐색
    candidates = []
    if "__file__" in globals():
        candidates.append(Path(__file__).resolve())
    candidates.append(Path.cwd().resolve())
    for base in candidates:
        for parent in [base] + list(base.parents):
            if (parent / "verl").exists():
                REPO_ROOT = parent
                break
        else:
            continue
        break

train_path = REPO_ROOT / "data" / "train_critique.parquet"
# verl 패키지를 로컬 경로에서 import할 수 있도록 추가
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))


# reward 서버 환경 변수 (모듈 import 전에 설정)
os.environ.setdefault("REWARD_PORT", "8000")
os.environ.setdefault("REWARD_API_URL", f"http://localhost:{os.environ['REWARD_PORT']}/v1")
os.environ.setdefault("REWARD_MODEL_NAME", "meta-llama/Llama-3.2-3B-Instruct")
os.environ.setdefault("REWARD_MODEL_PATH", os.environ["REWARD_MODEL_NAME"])
os.environ.setdefault("REWARD_TEMPERATURE", "0.6")
os.environ.setdefault("REWARD_TOP_P", "0.9")
os.environ.setdefault("REWARD_MAX_NEW_TOKENS", "2048")
os.environ.setdefault("REWARD_MAX_PROMPT_CHARS", "8192")

print("Repo root:", REPO_ROOT)
print("Train file:", train_path)
print("Reward API:", os.environ["REWARD_API_URL"])


Repo root: /data1/home/yunhochoi/verl
Train file: /data1/home/yunhochoi/verl/data/train_critique.parquet
Reward API: http://localhost:8000/v1


## Reward 서버(vLLM) 준비

이미 vLLM 서버가 떠 있다면 이 셀을 건너뛰고 `START_SERVER=False`로 두세요. 새로 띄우려면 `START_SERVER=True`로 변경하세요.


In [2]:
import subprocess
import time
import requests

START_SERVER = True  # True 로 바꾸면 아래에서 vLLM 서버를 띄움
reward_port = int(os.environ["REWARD_PORT"])
health_url = f"http://localhost:{reward_port}/health"
reward_model = os.environ["REWARD_MODEL_NAME"]
reward_gpu = os.environ.get("REWARD_GPU_ID", "2")
server_proc = None

if START_SERVER:
    try:
        requests.get(health_url, timeout=2).raise_for_status()
        print("이미 reward 서버가 실행 중입니다.")
    except Exception:
        cmd = [
            "python3", "-m", "vllm.entrypoints.openai.api_server",
            "--model", reward_model,
            "--port", str(reward_port),
            "--gpu-memory-utilization", "0.4",
            "--max-model-len", "10240",
            "--dtype", "bfloat16",
            "--tensor-parallel-size", "1",
        ]
        env = {**os.environ, "CUDA_VISIBLE_DEVICES": reward_gpu}
        print("Launching vLLM server...", cmd)
        server_proc = subprocess.Popen(cmd, env=env, stdout=open("vllm_server.log", "w"), stderr=subprocess.STDOUT)

        for wait_s in range(0, 310, 5):
            try:
                requests.get(health_url, timeout=2).raise_for_status()
                print("Reward 서버가 준비되었습니다.")
                break
            except Exception:
                time.sleep(5)
        else:
            raise RuntimeError("vLLM 서버가 시작되지 않았습니다. vllm_server.log를 확인하세요.")
else:
    print(f"START_SERVER=False -> 이미 떠 있는 http://localhost:{reward_port} 를 사용합니다.")


Launching vLLM server... ['python3', '-m', 'vllm.entrypoints.openai.api_server', '--model', 'meta-llama/Llama-3.2-3B-Instruct', '--port', '8000', '--gpu-memory-utilization', '0.4', '--max-model-len', '10240', '--dtype', 'bfloat16', '--tensor-parallel-size', '1']
Reward 서버가 준비되었습니다.


## 샘플 데이터 적재
훈련/평가에 쓰인 critique 데이터에서 몇 개 샘플을 뽑습니다.


In [6]:
df = pd.read_parquet("/data1/home/yunhochoi/verl/data/train_critique.parquet")
print("전체 샘플 수:", len(df))

# 테스트용으로 N개만 사용
N_SAMPLES = 1
samples = df.sample(n=N_SAMPLES, random_state=0).reset_index(drop=True)
# ground_truth는 reward_model['ground_truth'] 안에 JSON 문자열로 들어 있음
print(samples[["data_source"]].head())


전체 샘플 수: 1564
         data_source
0  critique_variants


## Qwen으로 critique 생성
Qwen2.5-7B-Instruct를 로드해 각 샘플의 `prompt`(대화 형식)에 대해 critique을 생성합니다.


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os
import asyncio
# verl 관련 import는 사용자 환경에 맞춰 유지
from verl.trainer.ppo.custom_rewards import critique_reward as cr
from verl.utils.reward_score.math_verify import compute_score as mv_score

# --- 설정 및 모델 로드 (기존과 동일) ---
CRITIQUE_MODEL = os.environ.get("CRITIQUE_MODEL_PATH", "Qwen/Qwen2.5-7B-Instruct")
LOAD_QWEN = True  
cr.NUM_REPEATS = 1
N_SAMPLES = 1

if LOAD_QWEN:
    tokenizer = AutoTokenizer.from_pretrained(CRITIQUE_MODEL)
    model = AutoModelForCausalLM.from_pretrained(
        CRITIQUE_MODEL,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
else:
    tokenizer = model = None

def generate_critique(messages, max_new_tokens=1024, temperature=0.7, top_p=0.9):
    if tokenizer is None or model is None:
        raise RuntimeError("LOAD_QWEN=True 로 설정하고 모델을 로드하세요.")

    prompt_text = tokenizer.apply_chat_template(
        list(messages), tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )

    gen_ids = output[0][inputs["input_ids"].shape[1]:]
    generated = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    return generated, prompt_text


critique_payloads = []
for i, row in samples.iterrows():
    messages = list(row["prompt"])  # numpy array -> list
    critique_text, rendered_prompt = generate_critique(messages)
    critique_payloads.append(
        {
            "idx": i,
            "messages": messages,
            "critique": critique_text,
            "rendered_prompt": rendered_prompt,
            "ground_truth": row["reward_model"]["ground_truth"],
        }
    )
    print(f"[{i}] critique (앞부분):", critique_text[:200], "...")

samples = df.sample(n=N_SAMPLES, random_state=0).reset_index(drop=True)

TARGET_VARIANT_IDX = 0


# === [수정된 핵심 부분] ===
def score_with_outputs(critique: str, ground_truth: str, sample_idx: int):
    runner = cr._get_runner()

    # 1. 비동기 함수: LLM 호출하여 '답변 텍스트'만 가져옴 (채점 X)
    async def _inner_collect():
        meta = json.loads(ground_truth)
        original_q = meta.get("original_question") or meta.get("question", "")
        original_traj = meta.get("original_trajectory") or meta.get("trajectory", "")
        variants = meta.get("variants", []) or []

        if not variants:
            return None

        var = variants[TARGET_VARIANT_IDX]
        var_q = var.get("q") or var.get("question")
        var_a = var.get("a") or var.get("answer")
        if not var_q or not var_a:
            return None

        sem = asyncio.Semaphore(cr.MAX_CONCURRENCY)
        prompt = cr._build_prompt(original_q, original_traj, critique, var_q)
        
        print(f"\n[Sample {sample_idx} / Variant {TARGET_VARIANT_IDX}] prompt_len={len(prompt)}")
        print("Q:", var_q)
        print("GT:", var_a)

        # 여기서 LLM 호출 (백그라운드 스레드 OK)
        preds = await cr._call_llama(runner.session, sem, prompt, n=1)
        print("preds:", preds)

        # 채점하지 않고 데이터만 리턴
        return {
            "var_q": var_q,
            "var_a": var_a,
            "preds": preds
        }

    # 2. runner로 실행하여 결과 받아오기
    # _inner_collect는 백그라운드 스레드에서 돌고, result는 여기서 받음
    result_data = runner.run(_inner_collect())

    # 3. 메인 스레드에서 채점 (mv_score) 수행
    final_score = 0.0
    details = []

    if result_data:
        var_q = result_data["var_q"]
        var_a = result_data["var_a"]
        preds = result_data["preds"]
        scores = []

        if preds:
            # 여기가 핵심: 메인 스레드에서 루프를 돌며 채점
            scores = [mv_score(p, var_a) for p in preds]
            print("scores:", scores)
            variant_score = sum(scores) / len(scores)
            final_score = variant_score
        else:
            variant_score = None
            final_score = 0.0

        details = [{
            "question": var_q,
            "answer": var_a,
            "predictions": preds,
            "scores": scores,
            "variant_score": variant_score,
        }]
    else:
        # variants가 없거나 에러인 경우
        details = [{"error": "No variants or invalid data"}]

    print(f"[Sample {sample_idx}] final_score={final_score}\n")
    return {"final_score": final_score, "details": details}

# 실행
scored = []
for payload in critique_payloads:
    result = score_with_outputs(payload["critique"], payload["ground_truth"], payload["idx"])
    scored.append({**payload, **result})

# 요약 출력
for item in scored:
    print(f"=== Sample {item['idx']} | final_score={item['final_score']:.4f}")
    for d in item["details"]:
        print(f"- Variant: {d['question']}")
        print("  preds:", d["predictions"])
        print("  scores:", d["scores"])
        print("  variant_score:", d["variant_score"])

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


[0] critique (앞부분): The provided model solution trace is correct and logically sound. Let's break down the steps to ensure clarity and accuracy:

1. **Identify the coefficients**:
   - The quadratic equation is \(8x^2 -  ...

[Sample 0 / Variant 0] prompt_len=2240
Q: What is the discriminant of $8x^2 - 48x + 69$?
GT: 96
preds: ['### Solution to the Variation Problem\n\nThe given quadratic equation is $8x^2 - 48x + 69$.\n\n**Step 1: Identify the coefficients**\n- $a = 8$\n- $b = -48$\n- $c = 69$\n\n**Step 2: Apply the discriminant formula**\n- The discriminant $\\Delta$ of a quadratic equation $ax^2 + bx + c$ is given by $\\Delta = b^2 - 4ac$.\n\n**Step 3: Substitute the values**\n- Substitute $a = 8$, $b = -48$, and $c = 69$ into the formula:\n  \\[\n  \\Delta = (-48)^2 - 4 \\cdot 8 \\cdot 69\n  \\]\n\n**Step 4: Calculate each part**\n- Calculate $(-48)^2$:\n  \\[\n  (-48)^2 = 2304\n  \\]\n- Calculate $4 \\cdot 8 \\cdot 69$:\n  \\[\n  4 \\cdot 8 = 32\n  \\]\n  \\[\n  32 \\cdot 69 = 220

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from pathlib import Path
import sys
import asyncio
from verl.trainer.ppo.custom_rewards import critique_reward as cr
from verl.utils.reward_score.math_verify import compute_score as mv_score

CRITIQUE_MODEL = os.environ.get("CRITIQUE_MODEL_PATH", "Qwen/Qwen2.5-7B-Instruct")
LOAD_QWEN = True  
cr.NUM_REPEATS = 1
N_SAMPLES = 1

if LOAD_QWEN:
    tokenizer = AutoTokenizer.from_pretrained(CRITIQUE_MODEL)
    model = AutoModelForCausalLM.from_pretrained(
        CRITIQUE_MODEL,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
else:
    tokenizer = model = None


def generate_critique(messages, max_new_tokens=1024, temperature=0.7, top_p=0.9):
    '''주어진 메시지 리스트로 critique 텍스트를 생성합니다.'''
    if tokenizer is None or model is None:
        raise RuntimeError("LOAD_QWEN=True 로 설정하고 모델을 로드하세요.")

    prompt_text = tokenizer.apply_chat_template(
        list(messages), tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
        )

    gen_ids = output[0][inputs["input_ids"].shape[1]:]
    generated = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    return generated, prompt_text


critique_payloads = []
for i, row in samples.iterrows():
    messages = list(row["prompt"])  # numpy array -> list
    critique_text, rendered_prompt = generate_critique(messages)
    critique_payloads.append(
        {
            "idx": i,
            "messages": messages,
            "critique": critique_text,
            "rendered_prompt": rendered_prompt,
            "ground_truth": row["reward_model"]["ground_truth"],
        }
    )
    print(f"[{i}] critique (앞부분):", critique_text[:200], "...")

samples = df.sample(n=N_SAMPLES, random_state=0).reset_index(drop=True)

TARGET_VARIANT_IDX = 0

def score_with_outputs(critique: str, ground_truth: str, sample_idx: int):
    runner = cr._get_runner()

    async def _inner():
        meta = json.loads(ground_truth)
        original_q = meta.get("original_question") or meta.get("question", "")
        original_traj = meta.get("original_trajectory") or meta.get("trajectory", "")
        variants = meta.get("variants", []) or []

        if not variants:
            return {"final_score": 0.0, "details": []}

        var = variants[TARGET_VARIANT_IDX]
        var_q = var.get("q") or var.get("question")
        var_a = var.get("a") or var.get("answer")
        if not var_q or not var_a:
            return {"final_score": 0.0, "details": []}

        sem = asyncio.Semaphore(cr.MAX_CONCURRENCY)
        prompt = cr._build_prompt(original_q, original_traj, critique, var_q)
        print(f"\n[Sample {sample_idx} / Variant {TARGET_VARIANT_IDX}] prompt_len={len(prompt)}")
        print("Q:", var_q)
        print("GT:", var_a)

        preds = await cr._call_llama(runner.session, sem, prompt, n=1)
        print("preds:", preds)

        if preds:
            scores = [mv_score(p, var_a) for p in preds]
            print("scores:", scores)
            variant_score = sum(scores) / len(scores)
            final_score = variant_score
        else:
            scores = []
            variant_score = None
            final_score = 0.0

        details = [{
            "question": var_q,
            "answer": var_a,
            "predictions": preds,
            "scores": scores,
            "variant_score": variant_score,
        }]
        print(f"[Sample {sample_idx}] final_score={final_score}\n")
        return {"final_score": final_score, "details": details}

    return runner.run(_inner())

# 실행
scored = []
for payload in critique_payloads:
    result = score_with_outputs(payload["critique"], payload["ground_truth"], payload["idx"])
    scored.append({**payload, **result})

# 요약 출력
for item in scored:
    print(f"=== Sample {item['idx']} | final_score={item['final_score']:.4f}")
    for d in item["details"]:
        print(f"- Variant: {d['question']}")
        print("  preds:", d["predictions"])
        print("  scores:", d["scores"])
        print("  variant_score:", d["variant_score"])


In [ ]:
cr.NUM_REPEATS = 1 

runner = cr._get_runner()
meta = json.loads(row["reward_model"]["ground_truth"])
original_q = meta["original_question"]
original_traj = meta["original_trajectory"]
var = meta["variants"][0]
var_q = var.get("q") or var.get("question")
var_a = var.get("a") or var.get("answer")

prompt = cr._build_prompt(original_q, original_traj, critique_payloads[0]["critique"], var_q)
print("prompt length:", len(prompt))
print("prompt preview:\n", prompt, "...")

preds = runner.run(cr._call_llama(runner.session, asyncio.Semaphore(cr.MAX_CONCURRENCY), prompt, n=1))
print("preds:", preds)

print("GT(answer):", var_a)
for i, p in enumerate(preds):
    s = mv_score(p, var_a)
    print(f"[{i}] score={s:.4f} pred preview:", p[:300], "...")

In [ ]:
meta = json.loads(row["reward_model"]["ground_truth"])
print(meta)

{'original_question': 'What is the discriminant of $8x^2 - 48x + 69$?', 'original_trajectory': 'The discriminant of a quadratic equation in the form $ax^2 + bx + c$ is given by the formula $b^2 - 4ac$.\n\nFor the given quadratic equation $8x^2 - 48x + 69$, we have:\n\n$a = 8$\n$b = -48$\n$c = 69$\n\nNow, we can plug these values into the formula for the discriminant:\n\n$b^2 - 4ac = (-48)^2 - 4(8)(69)$\n= 2304 - 2208\n= 96\n\nTherefore, the discriminant of $8x^2 - 48x + 69$ is 96.', 'variants': [{'snapshot': 'Oct-2023', 'question': 'What is the discriminant of $8x^2 - 48x + 69$?', 'answer': '96'}, {'snapshot': 'Nov-2023', 'question': 'What is the discriminant of $4x^2 - 66x + 81$?', 'answer': '3060'}, {'snapshot': 'Dec-2023', 'question': 'What is the discriminant of $2x^2 - 81x + 39$?', 'answer': '6249'}]}


: 

## vLLM 서버 정리 (옵션)
노트북에서 서버를 띄웠다면 종료합니다.


In [ ]:
if server_proc:
    server_proc.terminate()
    server_proc.wait()
    print("vLLM reward 서버를 종료했습니다.")
else:
    print("별도 종료할 서버가 없습니다.")


vLLM reward 서버를 종료했습니다.


: 